In [30]:
import pandas as pd
from IPython.display import display,HTML

table = """<table border="1" width="757">
<colgroup>
<col width="163">
<col width="99">
<col width="99">
<col width="99">
<col width="99">
<col width="99">
<col width="99">
</colgroup>
<thead>
<tr>
  <th width="163" height="50" rowspan="2" align="CENTER">구분</th>
  <th width="297" height="20" colspan="3" align="CENTER">당기</th>
  <th width="297" height="20" colspan="3" align="CENTER">전기</th>
</tr>
<tr>
  <th width="99" height="20" align="CENTER">토지</th>
  <th width="99" height="20" align="CENTER">건물</th>
  <th width="99" height="20" align="CENTER">합계</th>
  <th width="99" height="20" align="CENTER">토지</th>
  <th width="99" height="20" align="CENTER">건물</th>
  <th width="99" height="20" align="CENTER">합계</th>
</tr>
</thead>
<tbody>
<tr>
  <td width="163" height="20" valign="TOP">기초장부금액</td>
  <td width="99" height="20" align="RIGHT" valign="TOP">261,312</td>
  <td width="99" height="20" align="RIGHT" valign="TOP">4,186</td>
  <td width="99" height="20" align="RIGHT" valign="TOP">265,498</td>
  <td width="99" height="20" align="RIGHT" valign="TOP">257,770</td>
  <td width="99" height="20" align="RIGHT" valign="TOP">4,673</td>
  <td width="99" height="20" align="RIGHT" valign="TOP">262,443</td>
</tr>
<tr>
  <td width="163" height="20" valign="TOP">처분</td>
  <td width="99" height="20" align="RIGHT" valign="TOP">(109,625)</td>
  <td width="99" height="20" align="RIGHT" valign="TOP">(1,412)</td>
  <td width="99" height="20" align="RIGHT" valign="TOP">(111,037)</td>
  <td width="99" height="20" align="RIGHT" valign="TOP">(44)</td>
  <td width="99" height="20" align="RIGHT" valign="TOP">(66)</td>
  <td width="99" height="20" align="RIGHT" valign="TOP">(110)</td>
</tr>
<tr>
  <td width="163" height="20" valign="TOP">공정가치 평가</td>
  <td width="99" height="20" align="RIGHT" valign="TOP">7,664</td>
  <td width="99" height="20" align="RIGHT" valign="TOP">(71)</td>
  <td width="99" height="20" align="RIGHT" valign="TOP">7,593</td>
  <td width="99" height="20" align="RIGHT" valign="TOP">3,586</td>
  <td width="99" height="20" align="RIGHT" valign="TOP">(421)</td>
  <td width="99" height="20" align="RIGHT" valign="TOP">3,165</td>
</tr>
<tr>
  <td width="163" height="20" valign="TOP">기타</td>
  <td width="99" height="20" align="RIGHT" valign="TOP">4,500</td>
  <td width="99" height="20" align="RIGHT" valign="TOP">-</td>
  <td width="99" height="20" align="RIGHT" valign="TOP">4,500</td>
  <td width="99" height="20" align="RIGHT" valign="TOP">-</td>
  <td width="99" height="20" align="RIGHT" valign="TOP">-</td>
  <td width="99" height="20" align="RIGHT" valign="TOP">-</td>
</tr>
<tr>
  <td width="163" height="20" valign="TOP">기말장부금액</td>
  <td width="99" height="20" align="RIGHT" valign="TOP">163,851</td>
  <td width="99" height="20" align="RIGHT" valign="TOP">2,703</td>
  <td width="99" height="20" align="RIGHT" valign="TOP">166,554</td>
  <td width="99" height="20" align="RIGHT" valign="TOP">261,312</td>
  <td width="99" height="20" align="RIGHT" valign="TOP">4,186</td>
  <td width="99" height="20" align="RIGHT" valign="TOP">265,498</td>
</tr>
</tbody>
</table>"""

data = pd.read_html(table)

display(data[0])

,구분,당기,전기,Unnamed: 3_level_0,Unnamed: 4_level_0,Unnamed: 5_level_0,Unnamed: 6_level_0
,토지,건물,합계,토지,건물,합계,Unnamed: 6_level_1
0,기초장부금액,261312,4186,265498,257770,4673,262443
1,처분,"(109,625)","(1,412)","(111,037)",(44),(66),(110)
2,공정가치 평가,7664,(71),7593,3586,(421),3165
3,기타,4500,-,4500,-,-,-
4,기말장부금액,163851,2703,166554,261312,4186,265498


In [82]:
def pre_process_table(table):
    """
    INPUT:
        1. table - a bs4 element that contains the desired table: ie <table> ... </table>
    OUTPUT:
        a tuple of: 
            1. rows - a list of table rows ie: list of <tr>...</tr> elements
            2. num_rows - number of rows in the table
            3. num_cols - number of columns in the table
    Options:
        include_td_head_count - whether to use only th or th and td to count number of columns (default: False)
    """
    rows = [x for x in table.find_all('tr')]

    num_rows = len(rows)

    # get an initial column count. Most often, this will be accurate
    num_cols = max([len(x.find_all(['th','td'])) for x in rows])

    # sometimes, the tables also contain multi-colspan headers. This accounts for that:
    header_rows_set = [x.find_all(['th', 'td']) for x in rows if len(x.find_all(['th', 'td']))>num_cols/2]

    num_cols_set = []

    for header_rows in header_rows_set:
        num_cols = 0
        for cell in header_rows:
            row_span, col_span = get_spans(cell)
            num_cols+=len([cell.getText()]*col_span)

        num_cols_set.append(num_cols)

    num_cols = max(num_cols_set)

    return (rows, num_rows, num_cols)


def get_spans(cell):
        """
        INPUT:
            1. cell - a <td>...</td> or <th>...</th> element that contains a table cell entry
        OUTPUT:
            1. a tuple with the cell's row and col spans
        """
        if cell.has_attr('rowspan'):
            rep_row = int(cell.attrs['rowspan'])
        else: # ~cell.has_attr('rowspan'):
            rep_row = 1
        if cell.has_attr('colspan'):
            rep_col = int(cell.attrs['colspan'])
        else: # ~cell.has_attr('colspan'):
            rep_col = 1 

        return (rep_row, rep_col)

def process_rows(rows, num_rows, num_cols):
    """
    INPUT:
        1. rows - a list of table rows ie <tr>...</tr> elements
    OUTPUT:
        1. data - a Pandas dataframe with the html data in it
    """
    data = pd.DataFrame(np.ones((num_rows, num_cols))*np.nan)
    tab = 0
    tablist = []
    temptablist = []
    for i, row in enumerate(rows):
        try:
            col_stat = data.iloc[i,:][data.iloc[i,:].isnull()].index[0]
        except IndexError:
            print(i, row)
        tabs = []
        
        for j, cell in enumerate(row.find_all(['td', 'th'])):
            rep_row, rep_col = get_spans(cell)
            tabs.append((rep_row,rep_col))
            #print(rep_row, rep_col)
            #print("cols {0} to {1} with rep_col={2}".format(col_stat, col_stat+rep_col, rep_col))
            #print("\trows {0} to {1} with rep_row={2}".format(i, i+rep_row, rep_row))

            #find first non-na col and fill that one
            while any(data.iloc[i,col_stat+tab:col_stat+tab+rep_col].notnull()):
                col_stat+=1
            # data.iloc[i:i+rep_row,col_stat:col_stat+rep_col] = cell.getText()
            data.iloc[i,col_stat+tab] = cell.getText().lstrip().replace("\n","")
            
            if col_stat<data.shape[1]-1:
                col_stat+=rep_col
        
        tabcol = 1
        for t in tablist:
            if t[0]>1:
                temptablist.append((t[0]-1,t[1]))
                if t[1]>tabcol:
                    tabcol=t[1]
        tablist = temptablist
        temptablist = []
        
        tempt = (1,1)
        for t in tabs:
            if t[0]>tempt[0]:
                tempt = t
            
        if tempt[0]>1:
            tablist.append((tempt[0]-1,tempt[1]))
            if tempt[1]>tabcol:
                tabcol =tempt[1]
        
        tab = len(tablist)+tabcol-1
            

    return data

def main(table):
    rows, num_rows, num_cols = pre_process_table(table)
    df = process_rows(rows, num_rows, num_cols)
    return(df)

In [83]:
from bs4 import BeautifulSoup
import numpy as np

table = """
<table border="1" width="1003">
<colgroup>
<col width="126">
<col width="182">
<col width="116">
<col width="116">
<col width="116">
<col width="116">
<col width="116">
<col width="115">
</colgroup>
<thead>
<tr>
  <th width="308" height="46" colspan="2" rowspan="2" align="CENTER" style="font-size:10pt;">구분</th>
  <th width="348" height="18" colspan="3" align="CENTER" style="font-size:10pt;">당기</th>
  <th width="347" height="18" colspan="3" align="CENTER" style="font-size:10pt;">전기</th>
</tr>
<tr>
  <th width="116" height="18" align="CENTER" style="font-size:10pt;">1우선주</th>
  <th width="116" height="18" align="CENTER" style="font-size:10pt;">2우선주</th>
  <th width="116" height="18" align="CENTER" style="font-size:10pt;">보통주</th>
  <th width="116" height="18" align="CENTER" style="font-size:10pt;">1우선주</th>
  <th width="116" height="18" align="CENTER" style="font-size:10pt;">2우선주</th>
  <th width="115" height="18" align="CENTER" style="font-size:10pt;">보통주</th>
</tr>
</thead>
<tbody>
<tr>
  <td width="308" height="18" colspan="2" valign="TOP" style="font-size:10pt;">주당 액면금액(원)</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">5,000</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">5,000</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">5,000</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">5,000</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">5,000</td>
  <td width="115" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">5,000</td>
</tr>
<tr>
  <td width="126" height="214" rowspan="8" valign="TOP" style="font-size:10pt;">중간배당</td>
  <td width="182" height="18" valign="TOP" style="font-size:10pt;">발행주식수</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">4,411,074</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">985,685</td>
  <td width="115" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">20,885,121</td>
</tr>
<tr>
  <td width="182" height="18" valign="TOP" style="font-size:10pt;">자기주식수</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">(620,812)</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">(52,242)</td>
  <td width="115" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">(4,690,518)</td>
</tr>
<tr>
  <td width="182" height="18" valign="TOP" style="font-size:10pt;">배당받을 주식수</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;"> 3,790,262</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;"> 933,443</td>
  <td width="115" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;"> 16,194,603</td>
</tr>
<tr>
  <td width="182" height="18" valign="TOP" style="font-size:10pt;">액면배당률</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">10%</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">10%</td>
  <td width="115" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">10%</td>
</tr>
<tr>
  <td width="182" height="18" valign="TOP" style="font-size:10pt;">주당배당금(원)</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">500</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;"><span>500</span><br>
</td>
  <td width="115" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;"><span>500</span><br>
</td>
</tr>
<tr>
  <td width="182" height="18" valign="TOP" style="font-size:10pt;">배당액</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">1,895</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">467</td>
  <td width="115" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">8,097</td>
</tr>
<tr>
  <td width="182" height="18" valign="TOP" style="font-size:10pt;">주가(원)(주1)</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">68,060</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">65,740</td>
  <td width="115" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">124,300</td>
</tr>
<tr>
  <td width="182" height="18" valign="TOP" style="font-size:10pt;">배당수익률(주당배당금/주가)</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">0.73%</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">0.76%</td>
  <td width="115" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">0.40%</td>
</tr>
<tr>
  <td width="126" height="214" rowspan="8" valign="TOP" style="font-size:10pt;">기말배당예정</td>
  <td width="182" height="18" valign="TOP" style="font-size:10pt;">발행주식수</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">4,411,074</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">985,685</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">21,270,888</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">4,411,074</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">985,685</td>
  <td width="115" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">21,270,888</td>
</tr>
<tr>
  <td width="182" height="18" valign="TOP" style="font-size:10pt;">자기주식수</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">(620,812)</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">(52,242)</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">(5,977,528)</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">(620,812)</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">(52,242)</td>
  <td width="115" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">(5,401,098)</td>
</tr>
<tr>
  <td width="182" height="18" valign="TOP" style="font-size:10pt;">배당받을 주식수</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">3,790,262</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">933,443</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">15,293,360</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">3,790,262</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">933,443</td>
  <td width="115" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">15,869,790</td>
</tr>
<tr>
  <td width="182" height="18" valign="TOP" style="font-size:10pt;">액면배당률</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">92%</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">91%</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">91%</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">71%</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">70%</td>
  <td width="115" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">70%</td>
</tr>
<tr>
  <td width="182" height="18" valign="TOP" style="font-size:10pt;">주당배당금(원)</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">4,600</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">4,550</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">4,550</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">3,550</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">3,500</td>
  <td width="115" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">3,500</td>
</tr>
<tr>
  <td width="182" height="18" valign="TOP" style="font-size:10pt;">배당(예정)액</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">17,435</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">4,247</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">69,585</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">13,456</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">3,267</td>
  <td width="115" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">55,544</td>
</tr>
<tr>
  <td width="182" height="18" valign="TOP" style="font-size:10pt;">주가(원)(주1)</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">61,300</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">61,175</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">95,050</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">64,075</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">63,200</td>
  <td width="115" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">112,000</td>
</tr>
<tr>
  <td width="182" height="18" valign="TOP" style="font-size:10pt;">배당수익률(주당배당금/주가)</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">7.50%</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">7.44%</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">4.79%</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">5.46%</td>
  <td width="116" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">5.62%</td>
  <td width="115" height="18" align="RIGHT" valign="TOP" style="font-size:10pt;">3.13%</td>
</tr>
</tbody>
</table>
"""

In [84]:
soup = BeautifulSoup(table, 'lxml')
df=main(soup)
df

,0,1,2,3,4,5,6,7
0,구분,NaN,당기,NaN,NaN,전기,NaN,NaN
1,NaN,NaN,1우선주,2우선주,보통주,1우선주,2우선주,보통주
2,주당 액면금액(원),NaN,"5,000","5,000","5,000","5,000","5,000","5,000"
3,중간배당,발행주식수,-,-,-,"4,411,074","985,685","20,885,121"
4,NaN,자기주식수,-,-,-,"(620,812)","(52,242)","(4,690,518)"
5,NaN,배당받을 주식수,-,-,-,"3,790,262","933,443","16,194,603"
6,NaN,액면배당률,-,-,-,10%,10%,10%
7,NaN,주당배당금(원),-,-,-,500,500,500
8,NaN,배당액,-,-,-,"1,895",467,"8,097"
9,NaN,주가(원)(주1),-,-,-,"68,060","65,740","124,300"


In [85]:
table = """
<table border="1" width="1086">
<colgroup>
<col width="76">
<col width="265">
<col width="75">
<col width="75">
<col width="75">
<col width="75">
<col width="75">
<col width="74">
<col width="74">
<col width="74">
<col width="74">
<col width="74">
</colgroup>
<thead>
<tr>
  <th width="341" height="73" colspan="2" rowspan="2" align="CENTER" style="font-size:10pt;">구분</th>
  <th width="449" height="20" colspan="6" align="CENTER" style="font-size:10pt;">당기</th>
  <th width="296" height="20" colspan="4" align="CENTER" style="font-size:10pt;">전기</th>
</tr>
<tr>
  <th width="75" height="43" align="CENTER" style="font-size:10pt;">매출</th>
  <th width="75" height="43" align="CENTER" style="font-size:10pt;">기타수익</th>
  <th width="75" height="43" align="CENTER" style="font-size:10pt;">투자부동산<br>등 매각</th>
  <th width="75" height="43" align="CENTER" style="font-size:10pt;">매입</th>
  <th width="75" height="43" align="CENTER" style="font-size:10pt;">기타비용</th>
  <th width="74" height="43" align="CENTER" style="font-size:10pt;">투자부동산 등 매입</th>
  <th width="74" height="43" align="CENTER" style="font-size:10pt;">매출</th>
  <th width="74" height="43" align="CENTER" style="font-size:10pt;">기타수익</th>
  <th width="74" height="43" align="CENTER" style="font-size:10pt;">매입</th>
  <th width="74" height="43" align="CENTER" style="font-size:10pt;">기타비용</th>
</tr>
</thead>
<tbody>
<tr>
  <td width="76" height="193" rowspan="6" valign="TOP" style="font-size:10pt;">종속기업</td>
  <td width="265" height="20" valign="TOP" style="font-size:10pt;">두산중공업과 그 종속기업</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">117,672</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">1,344</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">78,015</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">785</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">122,497</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">520</td>
</tr>
<tr>
  <td width="265" height="20" valign="TOP" style="font-size:10pt;">두산인프라코어와 그 종속기업</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">163,208</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">513</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">29,763</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">29,894</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">5,811</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">211,292</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">33,762</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">5,927</td>
</tr>
<tr>
  <td width="265" height="20" valign="TOP" style="font-size:10pt;">두산건설과 그 종속기업</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">25,167</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">331</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">4,961</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">7,872</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">16,566</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">25,968</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">192</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">5,978</td>
</tr>
<tr>
  <td width="265" height="20" valign="TOP" style="font-size:10pt;">두산엔진과 그 종속기업</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">11,406</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">2,921</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">10,471</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">2,819</td>
</tr>
<tr>
  <td width="265" height="43" valign="TOP" style="font-size:10pt;">산업차량 해외종속기업(Doosan Industrial Vehicle UK, America, Yantai 등)</td>
  <td width="75" height="43" align="RIGHT" valign="BOTTOM">
<p style="font-size:10pt;">171,887</p>
</td>
  <td width="75" height="43" align="RIGHT" valign="BOTTOM" style="font-size:10pt;">476</td>
  <td width="75" height="43" align="RIGHT" valign="BOTTOM" style="font-size:10pt;">-</td>
  <td width="75" height="43" align="RIGHT" valign="BOTTOM" style="font-size:10pt;">29,895</td>
  <td width="75" height="43" align="RIGHT" valign="BOTTOM" style="font-size:10pt;">486</td>
  <td width="74" height="43" align="RIGHT" valign="BOTTOM" style="font-size:10pt;">-</td>
  <td width="74" height="43" align="RIGHT" valign="BOTTOM" style="font-size:10pt;">164,384</td>
  <td width="74" height="43" align="RIGHT" valign="BOTTOM" style="font-size:10pt;">190</td>
  <td width="74" height="43" align="RIGHT" valign="BOTTOM" style="font-size:10pt;">32,059</td>
  <td width="74" height="43" align="RIGHT" valign="BOTTOM" style="font-size:10pt;">724</td>
</tr>
<tr>
  <td width="265" height="20" valign="TOP" style="font-size:10pt;">기타</td>
  <td width="75" height="20" align="RIGHT" valign="TOP">
<p style="font-size:10pt;">230,692</p>
</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">1,185</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">13,175</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">12,757</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">258,890</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">2,147</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">10,952</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">12,036</td>
</tr>
<tr>
  <td width="341" height="20" colspan="2" align="CENTER" valign="BOTTOM" style="font-size:10pt;">소계</td>
  <td width="75" height="20" align="RIGHT" valign="TOP">
<p style="font-size:10pt;">720,032</p>
</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">3,849</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">112,739</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">72,964</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">30,632</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">16,566</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">793,502</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">2,529</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">76,773</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">28,004</td>
</tr>
<tr>
  <td width="341" height="20" colspan="2" valign="TOP" style="font-size:10pt;">관계기업</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">374</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">3,399</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">74</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">20</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">38</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">55</td>
</tr>
<tr>
  <td width="341" height="20" colspan="2" valign="TOP" style="font-size:10pt;">공동기업</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">988</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">931</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
</tr>
<tr>
  <td width="341" height="20" colspan="2" valign="TOP" style="font-size:10pt;">기타특수관계자 </td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">11,274</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">39</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">17,591</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">12,320</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">-</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">1,692</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">8,312</td>
</tr>
<tr>
  <td width="341" height="20" colspan="2" align="CENTER" valign="TOP" style="font-size:10pt;">합계</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">732,668</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">3,849</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">112,739</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">76,402</td>
  <td width="75" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">48,297</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">16,566</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">806,773</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">2,529</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">78,503</td>
  <td width="74" height="20" align="RIGHT" valign="TOP" style="font-size:10pt;">36,371</td>
</tr>
</tbody>
</table>
"""

In [86]:
soup = BeautifulSoup(table, 'lxml')
df=main(soup)
df

,0,1,2,3,4,5,6,7,8,9,10,11
0,구분,NaN,당기,NaN,NaN,NaN,NaN,NaN,전기,NaN,NaN,NaN
1,NaN,NaN,매출,기타수익,투자부동산등 매각,매입,기타비용,투자부동산 등 매입,매출,기타수익,매입,기타비용
2,종속기업,두산중공업과 그 종속기업,"117,672","1,344","78,015",-,785,-,"122,497",-,-,520
3,NaN,두산인프라코어와 그 종속기업,"163,208",513,"29,763","29,894","5,811",-,"211,292",-,"33,762","5,927"
4,NaN,두산건설과 그 종속기업,"25,167",331,"4,961",-,"7,872","16,566","25,968",192,-,"5,978"
5,NaN,두산엔진과 그 종속기업,"11,406",-,-,-,"2,921",-,"10,471",-,-,"2,819"
6,NaN,"산업차량 해외종속기업(Doosan Industrial Vehicle UK, Amer...","171,887",476,-,"29,895",486,-,"164,384",190,"32,059",724
7,NaN,기타,"230,692","1,185",-,"13,175","12,757",-,"258,890","2,147","10,952","12,036"
8,소계,NaN,"720,032","3,849","112,739","72,964","30,632","16,566","793,502","2,529","76,773","28,004"
9,관계기업,NaN,374,-,-,"3,399",74,-,20,-,38,55


In [33]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

def pre_process_table(table):
    """
    INPUT:
        1. table - a bs4 element that contains the desired table: ie <table> ... </table>
    OUTPUT:
        a tuple of: 
            1. rows - a list of table rows ie: list of <tr>...</tr> elements
            2. num_rows - number of rows in the table
            3. num_cols - number of columns in the table
    Options:
        include_td_head_count - whether to use only th or th and td to count number of columns (default: False)
    """
    rows = [x for x in table.find_all('tr')]

    num_rows = len(rows)

    # get an initial column count. Most often, this will be accurate
    num_cols = max([len(x.find_all(['th','td'])) for x in rows])

    # sometimes, the tables also contain multi-colspan headers. This accounts for that:
    header_rows_set = [x.find_all(['th', 'td']) for x in rows if len(x.find_all(['th', 'td']))>num_cols/2]

    num_cols_set = []

    for header_rows in header_rows_set:
        num_cols = 0
        for cell in header_rows:
            row_span, col_span = get_spans(cell)
            num_cols+=len([cell.getText()]*col_span)

        num_cols_set.append(num_cols)

    num_cols = max(num_cols_set)

    return (rows, num_rows, num_cols)


def get_spans(cell):
        """
        INPUT:
            1. cell - a <td>...</td> or <th>...</th> element that contains a table cell entry
        OUTPUT:
            1. a tuple with the cell's row and col spans
        """
        if cell.has_attr('rowspan'):
            rep_row = int(cell.attrs['rowspan'])
        else: # ~cell.has_attr('rowspan'):
            rep_row = 1
        if cell.has_attr('colspan'):
            rep_col = int(cell.attrs['colspan'])
        else: # ~cell.has_attr('colspan'):
            rep_col = 1 

        return (rep_row, rep_col)

def process_rows(rows, num_rows, num_cols):
    """
    INPUT:
        1. rows - a list of table rows ie <tr>...</tr> elements
    OUTPUT:
        1. data - a Pandas dataframe with the html data in it
    """
    data = pd.DataFrame(np.ones((num_rows, num_cols))*np.nan)
    tab = 0
    tablist = []
    temptablist = []
    for i, row in enumerate(rows):
        try:
            col_stat = data.iloc[i,:][data.iloc[i,:].isnull()].index[0]
        except IndexError:
            print(i, row)
        tabs = []
        
        for j, cell in enumerate(row.find_all(['td', 'th'])):
            #print(i, j, cell)
            rep_row, rep_col = get_spans(cell)
            tabs.append((rep_row,rep_col))
            #print(rep_row, rep_col)
            #print("cols {0} to {1} with rep_col={2}".format(col_stat, col_stat+rep_col, rep_col))
            #print("\trows {0} to {1} with rep_row={2}".format(i, i+rep_row, rep_row))

            #find first non-na col and fill that one
            while any(data.iloc[i,col_stat+tab:col_stat+tab+rep_col].notnull()):
                col_stat+=1
            # data.iloc[i:i+rep_row,col_stat:col_stat+rep_col] = cell.getText()
            data.iloc[i,col_stat+tab] = cell.getText().lstrip().replace("\n","")
            
            if col_stat<data.shape[1]-1:
                col_stat+=rep_col
        
        tabcol = 1
        for t in tablist:
            if t[0]>1:
                temptablist.append((t[0]-1,t[1]))
                if t[1]>tabcol:
                    tabcol=t[1]
        tablist = temptablist[:]
        temptablist = []
        tempt = (1,1)
        for t in tabs:
            if t[0]>tempt[0]:
                tempt = t
            if t[0]>1:
                tablist.append((t[0]-1,tempt[1]))
                
        if tempt[0]>1:
            if tempt[1]>tabcol:
                tabcol =tempt[1]
#         print("tablist: {0}".format(tablist))
#         print("len(lablist): {0}".format(len(tablist)))
#         print("tabcol: {0}".format(tabcol))
        tab = len(tablist)+tabcol-1
            

    return data

def main(table):
    rows, num_rows, num_cols = pre_process_table(table)
    df = process_rows(rows, num_rows, num_cols)
    return(df)

In [34]:
table="""<TABLE border='1' width='823'>
<COLGROUP>
<COL width='119'></COL>
<COL width='282'></COL>
<COL width='211'></COL>
<COL width='211'></COL>
</COLGROUP>
<THEAD>
<TR>
  <TH width='823' height='20' colspan='4' align='RIGHT'>(단위:천원)</TH>
</TR>
<TR>
  <TH width='119' height='50' rowspan='2' align='CENTER'>특수관계구분</TH>
  <TH width='282' height='50' rowspan='2' align='CENTER'>회사명</TH>
  <TH width='422' height='20' colspan='2' align='CENTER'>현금출자</TH>
</TR>
<TR>
  <TH width='211' height='20' align='CENTER'>당 &nbsp; &nbsp;기</TH>
  <TH width='211' height='20' align='CENTER'>전 &nbsp; &nbsp;기</TH>
</TR>
</THEAD>
<TBODY>
<TR>
  <TD width='119' height='50' rowspan='2'>종속기업</TD>
  <TD width='282' height='20' valign='TOP'>Hyundai Steel Chongqing Co., Ltd.</TD>
  <TD width='211' height='20' align='RIGHT'>7,031,875</TD>
  <TD width='211' height='20' align='RIGHT'>3,377,100</TD>
</TR>
<TR>
  <TD width='282' height='20' valign='TOP'>Hyundai Steel Anantapur Private, Ltd.</TD>
  <TD width='211' height='20' align='RIGHT'>1,852</TD>
  <TD width='211' height='20' align='RIGHT'>-</TD>
</TR>
</TBODY>
</TABLE>
"""

soup = BeautifulSoup(table, 'lxml')
df=main(soup)
df

,0,1,2,3
0,(단위:천원),NaN,NaN,NaN
1,특수관계구분,회사명,현금출자,NaN
2,NaN,NaN,당 기,전 기
3,종속기업,"Hyundai Steel Chongqing Co., Ltd.","7,031,875","3,377,100"
4,NaN,"Hyundai Steel Anantapur Private, Ltd.","1,852",-


In [35]:
table="""
<TABLE border='1' width='1006'>
<COLGROUP>
<COL width='143'></COL>
<COL width='343'></COL>
<COL width='130'></COL>
<COL width='130'></COL>
<COL width='130'></COL>
<COL width='130'></COL>
</COLGROUP>
<THEAD>
<TR>
  <TH width='1006' height='20' colspan='6' align='RIGHT'>(단위:천원)</TH>
</TR>
<TR>
  <TH width='143' height='50' rowspan='2' align='CENTER'>특수관계구분</TH>
  <TH width='343' height='50' rowspan='2' align='CENTER'>회사명</TH>
  <TH width='260' height='20' colspan='2' align='CENTER'>채 &nbsp; &nbsp;권(*)</TH>
  <TH width='260' height='20' colspan='2' align='CENTER'>채 &nbsp; &nbsp;무</TH>
</TR>
<TR>
  <TH width='130' height='20' align='CENTER'>매출채권</TH>
  <TH width='130' height='20' align='CENTER'>기타채권</TH>
  <TH width='130' height='20' align='CENTER'>매입채무</TH>
  <TH width='130' height='20' align='CENTER'>기타채무</TH>
</TR>
</THEAD>
<TBODY>
<TR>
  <TD width='143' height='43'>유의적인 영향력을<BR/>행사하는 기업</TD>
  <TD width='343' height='43'>기아자동차(주)</TD>
  <TD width='130' height='43' align='RIGHT'>24,080,231</TD>
  <TD width='130' height='43' align='RIGHT'>-</TD>
  <TD width='130' height='43' align='RIGHT'>364,492</TD>
  <TD width='130' height='43' align='RIGHT'>17,357,120</TD>
</TR>
<TR>
  <TD width='143' height='560' rowspan='19'>종속기업</TD>
  <TD width='343' height='20'>현대비앤지스틸(주)</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
  <TD width='130' height='20' align='RIGHT'>229,326</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
</TR>
<TR>
  <TD width='343' height='20'>현대종합특수강(주)</TD>
  <TD width='130' height='20' align='RIGHT'>13,159,167</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
</TR>
<TR>
  <TD width='343' height='20'>Qingdao Hyundai Machinery Co., Ltd.</TD>
  <TD width='130' height='20' align='RIGHT'>12,047,765</TD>
  <TD width='130' height='20' align='RIGHT'>1,420</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
</TR>
<TR>
  <TD width='343' height='20'>Hyundai Steel Beijing Process Co., Ltd.</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
  <TD width='130' height='20' align='RIGHT'>116,475</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
  <TD width='130' height='20' align='RIGHT'>5,299</TD>
</TR>
<TR>
  <TD width='343' height='20'>Hyundai Steel Investment(China) Co., Ltd.</TD>
  <TD width='130' height='20' align='RIGHT'>14,526,131</TD>
  <TD width='130' height='20' align='RIGHT'>140,726</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
  <TD width='130' height='20' align='RIGHT'>28,482</TD>
</TR>
<TR>
  <TD width='343' height='20'>Hyundai Steel Jiangsu Process Co., Ltd.</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
  <TD width='130' height='20' align='RIGHT'>76,953</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
  <TD width='130' height='20' align='RIGHT'>5,690</TD>
</TR>
<TR>
  <TD width='343' height='20'>Hyundai Steel Suzhou Process Co., Ltd.</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
  <TD width='130' height='20' align='RIGHT'>45,028</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
</TR>
<TR>
  <TD width='343' height='20'>Hyundai Steel Tianjin Co., Ltd.</TD>
  <TD width='130' height='20' align='RIGHT'>10,692,425</TD>
  <TD width='130' height='20' align='RIGHT'>291,867</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
  <TD width='130' height='20' align='RIGHT'>2,256,999</TD>
</TR>
<TR>
  <TD width='343' height='20'>Hyundai Steel Chongqing Co., Ltd.</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
  <TD width='130' height='20' align='RIGHT'>43,002</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
</TR>
<TR>
  <TD width='343' height='20'>Hyundai Steel America, Inc.</TD>
  <TD width='130' height='20' align='RIGHT'>6,509,140</TD>
  <TD width='130' height='20' align='RIGHT'>122,032</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
  <TD width='130' height='20' align='RIGHT'>969</TD>
</TR>
<TR>
  <TD width='343' height='20'>Hyundai Steel USA, Inc.</TD>
  <TD width='130' height='20' align='RIGHT'>1,132,020</TD>
  <TD width='130' height='20' align='RIGHT'>7,309</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
</TR>
<TR>
  <TD width='343' height='20'>Hyundai Steel India Private, Ltd.</TD>
  <TD width='130' height='20' align='RIGHT'>14,205,705</TD>
  <TD width='130' height='20' align='RIGHT'>91,710</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
  <TD width='130' height='20' align='RIGHT'>69,353</TD>
</TR>
<TR>
  <TD width='343' height='20'>Hyundai Steel Pipe India Private, Ltd.</TD>
  <TD width='130' height='20' align='RIGHT'>2,470,323</TD>
  <TD width='130' height='20' align='RIGHT'>30,325</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
</TR>
<TR>
  <TD width='343' height='20'>Hyundai Steel Slovakia s.r.o.</TD>
  <TD width='130' height='20' align='RIGHT'>11,896,108</TD>
  <TD width='130' height='20' align='RIGHT'>67,296</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
  <TD width='130' height='20' align='RIGHT'>25,847</TD>
</TR>
<TR>
  <TD width='343' height='20'>Hyundai Steel TR Automotive Steel Parts Co., Ltd.</TD>
  <TD width='130' height='20' align='RIGHT'>1,753,629</TD>
  <TD width='130' height='20' align='RIGHT'>61,153</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
</TR>
<TR>
  <TD width='343' height='20'>Hyundai Steel Czech s.r.o.</TD>
  <TD width='130' height='20' align='RIGHT'>8,027,376</TD>
  <TD width='130' height='20' align='RIGHT'>72,116</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
  <TD width='130' height='20' align='RIGHT'>33,759</TD>
</TR>
<TR>
  <TD width='343' height='20'>Hyundai Steel Rus LLC</TD>
  <TD width='130' height='20' align='RIGHT'>1,093,216</TD>
  <TD width='130' height='20' align='RIGHT'>119,498</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
</TR>
<TR>
  <TD width='343' height='20'>Hyundai Steel Mexico S de R.L. de C.V</TD>
  <TD width='130' height='20' align='RIGHT'>6,950,402</TD>
  <TD width='130' height='20' align='RIGHT'>162,210</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
  <TD width='130' height='20' align='RIGHT'>14,974</TD>
</TR>
<TR>
  <TD width='343' height='20'>Hyundai Steel Industry and Trade Brazil LLC</TD>
  <TD width='130' height='20' align='RIGHT'>9,396,795</TD>
  <TD width='130' height='20' align='RIGHT'>127,903</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
  <TD width='130' height='20' align='RIGHT'>7,913</TD>
</TR>
<TR>
  <TD width='143' height='20'>관계기업</TD>
  <TD width='343' height='20'>현대그린파워(주)</TD>
  <TD width='130' height='20' align='RIGHT'>23,191,809</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
  <TD width='130' height='20' align='RIGHT'>3,065,326</TD>
  <TD width='130' height='20' align='RIGHT'>24,055</TD>
</TR>
<TR>
  <TD width='143' height='170' rowspan='6'>대규모기업집단 등</TD>
  <TD width='343' height='20'>현대자동차(주)</TD>
  <TD width='130' height='20' align='RIGHT'>83,069,452</TD>
  <TD width='130' height='20' align='RIGHT'>25,922,401</TD>
  <TD width='130' height='20' align='RIGHT'>317,225</TD>
  <TD width='130' height='20' align='RIGHT'>17,963,091</TD>
</TR>
<TR>
  <TD width='343' height='20'>현대건설(주)</TD>
  <TD width='130' height='20' align='RIGHT'>91,172,228</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
  <TD width='130' height='20' align='RIGHT'>2,284,095</TD>
</TR>
<TR>
  <TD width='343' height='20'>현대엔지니어링(주)</TD>
  <TD width='130' height='20' align='RIGHT'>12,226,282</TD>
  <TD width='130' height='20' align='RIGHT'>-</TD>
  <TD width='130' height='20' align='RIGHT'>869,878</TD>
  <TD width='130' height='20' align='RIGHT'>58,776,056</TD>
</TR>
<TR>
  <TD width='343' height='20'>현대로템(주)</TD>
  <TD width='130' height='20' align='RIGHT'>2,259,212</TD>
  <TD width='130' height='20' align='RIGHT'>2,036</TD>
  <TD width='130' height='20' align='RIGHT'>1,207,293</TD>
  <TD width='130' height='20' align='RIGHT'>142,326,449</TD>
</TR>
<TR>
  <TD width='343' height='20'>현대글로비스(주)</TD>
  <TD width='130' height='20' align='RIGHT'>1,708,602</TD>
  <TD width='130' height='20' align='RIGHT'>1,361</TD>
  <TD width='130' height='20' align='RIGHT'>30,352,471</TD>
  <TD width='130' height='20' align='RIGHT'>94,050,879</TD>
</TR>
<TR>
  <TD width='343' height='20'>기타</TD>
  <TD width='130' height='20' align='RIGHT'>22,257,047</TD>
  <TD width='130' height='20' align='RIGHT'>160,481,932</TD>
  <TD width='130' height='20' align='RIGHT'>15,737,509</TD>
  <TD width='130' height='20' align='RIGHT'>73,133,473</TD>
</TR>
</TBODY>
</TABLE>

"""

soup = BeautifulSoup(table, 'lxml')
df=main(soup)
df

,0,1,2,3,4,5
0,(단위:천원),NaN,NaN,NaN,NaN,NaN
1,특수관계구분,회사명,채 권(*),NaN,채 무,NaN
2,NaN,NaN,매출채권,기타채권,매입채무,기타채무
3,유의적인 영향력을행사하는 기업,기아자동차(주),"24,080,231",-,"364,492","17,357,120"
4,종속기업,현대비앤지스틸(주),-,-,"229,326",-
5,NaN,현대종합특수강(주),"13,159,167",-,-,-
6,NaN,"Qingdao Hyundai Machinery Co., Ltd.","12,047,765","1,420",-,-
7,NaN,"Hyundai Steel Beijing Process Co., Ltd.",-,"116,475",-,"5,299"
8,NaN,"Hyundai Steel Investment(China) Co., Ltd.","14,526,131","140,726",-,"28,482"
9,NaN,"Hyundai Steel Jiangsu Process Co., Ltd.",-,"76,953",-,"5,690"
